created by Ignacio Oguiza - email: oguiza@gmail.com

In [1]:
%%javascript
utils.load_extension('collapsible_headings/main')
utils.load_extension('hide_input/main')
utils.load_extension('autosavetime/main')
utils.load_extension('execute_time/ExecuteTime')
utils.load_extension('code_prettify/code_prettify')
utils.load_extension('scroll_down/main')
utils.load_extension('jupyter-js-widgets/extension')

<IPython.core.display.Javascript object>

In [2]:
from IPython.display import display, HTML, clear_output
display(HTML("<style>.container { width:100% !important; }</style>"))

<img src="./images/Rocket.svg"  width="150">

ROCKET (RandOm Convolutional KErnel Transform) is a new Time Series Classification (TSC) method that has just been released (Oct 29th, 2019), and has achieved **state-of-the-art performance on the UCR univariate time series classification datasets, surpassing HIVE-COTE (the previous state of the art since 2017) in accuracy, with exceptional speed compared to other traditional DL methods.** To achieve these 2 things at once is **VERY IMPRESSIVE**. ROCKET is certainly a new TSC method you should try.

Authors:
Dempster, A., Petitjean, F., & Webb, G. I. (2019). ROCKET: Exceptionally fast and accurate time series classification using random convolutional kernels. arXiv preprint arXiv:1910.13051.

[paper](https://arxiv.org/pdf/1910.13051)

There are 2 main limitations to the original ROCKET method though:
- The release code doesn't handle multivariate data
- It doesn't run on a GPU, so it's slow when used with a large datasets

In this notebook you will learn: 
- how you can use the original ROCKET method
- you will also learn about a new ROCKET version I have developed in Pytorch, that handles both univariate and multivariate data, and uses GPU
- you will see how you can integrate the ROCKET features with fastai or other classifiers

## Import libraries 📚

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from fastai_timeseries import *
print('pytorch:', torch.__version__)
print('fastai :', fastai.__version__)

pytorch: 1.3.0
fastai : 1.0.59


## How to use the original ROCKET method? 🚀

ROCKET is applied in 2 phases:

1. Generate features from each time series: ROCKET calculates 20k features from each time series, independently of the dimensions. 
2. Apply a classifier to those calculated features. Those features are then used by the classifier of your choice. In the original code they use 2 simple linear classifiers: RidgeClassifierCV and Logistic Regression, but you can use any classifier.

Let's first generate the features. We'll import data from a UCR Time Series dataset.

The original method requires the time series to be in a 2d array of dtype np.float64.

In [35]:
X_train, y_train, X_valid, y_valid = get_UCR_data('OliveOil')
seq_len = X_train.shape[-1]
X_train = X_train[:, 0].astype(np.float64)
X_valid = X_valid[:, 0].astype(np.float64)
X_train.shape, X_valid.shape

((30, 570), (30, 570))

Now we are normalize the data to mean 0 and std 1 per sample (recommended by the authors), that is they set each sample to mean 0 and std 1).

In [36]:
X_train = (X_train - X_train.mean(axis = 1, keepdims = True)) / (X_train.std(axis = 1, keepdims = True) + 1e-8)
X_valid = (X_valid - X_valid.mean(axis = 1, keepdims = True)) / (X_valid.std(axis = 1, keepdims = True) + 1e-8)
X_train.mean(axis = 1, keepdims = True).shape

(30, 1)

To generate the features, we first need to create the 10k random kernels that will be used to process the data.

In [37]:
kernels = generate_kernels(seq_len, 10000)

Now we apply those ramdom kernels to the data

In [38]:
X_train_tfm = apply_kernels(X_train, kernels)
X_valid_tfm = apply_kernels(X_valid, kernels)

So now we have the features, and we are ready to apply a classifier. 

Let's use simple, linear RidgeClassifierCV as they propose in the paper. We first instantiate it. 

Note:
alphas: Array of alpha values to try. Regularization strength; must be a positive float. Regularization improves the conditioning of the problem and reduces the variance of the estimates. Larger values specify stronger regularization. Alpha corresponds to C^-1 in other linear models such as LogisticRegression or LinearSVC.

In [39]:
from sklearn.linear_model import RidgeClassifierCV
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 7), normalize=True)

In [40]:
 classifier.fit(X_train_tfm, y_train)

RidgeClassifierCV(alphas=array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                  class_weight=None, cv=None, fit_intercept=True,
                  normalize=True, scoring=None, store_cv_values=False)

In [33]:
classifier.score(X_valid_tfm, y_valid)

0.9333333333333333

☣️ **This is pretty impressive! It matches or exceeds the state-of-the-art performance without any fine tuning in 1.5 seconds!!!**

In [41]:
kernels = generate_kernels(seq_len, 10000)
X_train_tfm = apply_kernels(X_train, kernels)
X_valid_tfm = apply_kernels(X_valid, kernels)
from sklearn.linear_model import RidgeClassifierCV
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 7), normalize=True)
classifier.fit(X_train_tfm, y_train)
classifier.score(X_valid_tfm, y_valid)

0.9333333333333333

Bear in mind that this process is not deterministic since there is randomness involved in the kernels. In thiis case, performance may vay between .9 to .933

## How to use ROCKET with large and/ or multivariate datasets on GPU? ❓

As stated before, the current ROCKET method doesn't provide multivariate time series or GPU support. This may be a drawback in many cases. To handle both limitations I've created a new version of ROCKET in Pytorch. 

In [5]:
import torch
import torch.nn as nn

class ROCKET(nn.Module):
    def __init__(self, c_in, seq_len, n_kernels=10000, kss=[7, 9, 11]):
        
        '''
        ROCKET is a GPU Pytorch implementation of the ROCKET methods generate_kernels 
        and apply_kernels that can be used 
        with univariate and multivariate time series.
        Input: is a 3d torch tensor of type torch.float32. When used with univariate TS, 
        make sure you transform the 2d to 3d by adding unsqueeze(1)
        c_in: number of channels in (features). For univariate c_in is 1.
        seq_len: sequence length (is the last dimension of the input)
        '''
        super().__init__()
        kss = [ks for ks in kss if ks < seq_len]
        convs = nn.ModuleList()
        for i in range(n_kernels):
            ks = np.random.choice(kss)
            dilation = 2**np.random.uniform(0, np.log2((seq_len - 1) // (ks - 1)))
            padding = int((ks - 1) * dilation // 2) if np.random.randint(2) == 1 else 0
            weight = torch.randn(1, c_in, ks)
            weight -= weight.mean()
            bias = 2 * (torch.rand(1) - .5)
            layer = nn.Conv1d(c_in, 1, ks, padding=2 * padding, dilation=int(dilation), bias=True)
            layer.weight = torch.nn.Parameter(weight, requires_grad=False)
            layer.bias = torch.nn.Parameter(bias, requires_grad=False)
            convs.append(layer)
        self.convs = convs
        self.n_kernels = n_kernels
        self.kss = kss

    def forward(self, x):
        for i in range(self.n_kernels):
            out = self.convs[i](x)
            _max = out.max(dim=-1).values
            _ppv = torch.gt(out, 0).sum(dim=-1).float() / out.shape[-1]
            cat = torch.cat((_max, _ppv), dim=-1)
            output = cat if i == 0 else torch.cat((output, cat), dim=-1)
        return output

The way to use ROCKET in Pytorch is the following:

First you prepare the input data. The input to ROCKET Pytorch is a 3d tensor (preferrable on gpu)

In [8]:
X_train, y_train, X_valid, y_valid = get_UCR_data('OliveOil')
_, features, seq_len = X_train.shape
X_train = (X_train - X_train.mean(axis = (1, 2), keepdims = True)) / (X_train.std(axis = (1, 2), keepdims = True) + 1e-8)
X_valid = (X_valid - X_valid.mean(axis = (1, 2), keepdims = True)) / (X_valid.std(axis = (1, 2), keepdims = True) + 1e-8)
X_train_tensor = torch.tensor(X_train, device=device)
X_valid_tensor = torch.tensor(X_valid, device=device)

Then you instantiate the Pytorch model.

In [9]:
model = ROCKET(features, seq_len, n_kernels=10000, kss=[7, 9, 11]).to(device)

In [10]:
X_train_tfm = model(X_train_tensor)
X_valid_tfm = model(X_valid_tensor)

### RidgeClassifierCV

And now you apply a classifier of your choice. 
With RidgeClassifierCV in particular, there's no need to normalize the extracted features before passing them to the classifier, as it does it internally (if normalize is set to True as recommended by the authors).

In [11]:
from sklearn.linear_model import RidgeClassifierCV
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 7), normalize=True)
classifier.fit(X_train_tfm, y_train)
classifier.score(X_valid_tfm, y_valid)

0.9333333333333333

### Logistic Regression

In the case of other classifiers (like Logistic Regression), the authors recommend a per-feature normalization.

In [16]:
from sklearn.linear_model import LogisticRegression
f_mean = X_train_tfm.mean(axis = 0, keepdims = True)
f_std = X_train_tfm.std(axis = 0, keepdims = True) + 1e-8
f_mean, f_std = 0, 1
X_train_tfm2 = (X_train_tfm - f_mean) / f_std
X_valid_tfm2 = (X_valid_tfm - f_mean) / f_std
for C in np.logspace(-3,3,7):
    classifier = LogisticRegression(penalty='l2', C=C, n_jobs=-1)
    classifier.fit(X_train_tfm2, y_train)
    print('{:.2E} {:.5f}'.format(C, classifier.score(X_valid_tfm2, y_valid)))

1.00E-03 0.40000
1.00E-02 0.46667
1.00E-01 0.66667
1.00E+00 0.76667
1.00E+01 0.83333
1.00E+02 0.83333
1.00E+03 0.83333


### Results

I've checked the performance for all 85 univariate datasets in the UCR archive, and performance is the same as the published one. Average performance accross all datasets is 85.100% (original) vs	85.085% (with the Pytorch version).

I've also run some tests to and compared performance (time) of ROCKET original code vs Pytorch. These are the results:

<img src="./images/Time_ROCKET_comp.png" width=800>

So, as you can see, in small datasets, the original method may be even faster than the GPU method, but for larger datasets, there's a great benefit in using the GPU version.

In addition to this, I have also run the code on the TSC UCR multivariate datasets (all the ones that don't caontain nan values), and the results are also very good, beating the previous state-of-the-art in this category as well by a large margin. For example, ROCKET reduces InceptionTime errors by 26% on average.

In [7]:
iterations = 5
datasets = sorted(get_UCR_multivariate_list())
n_kernels=10_000
kss=[7, 9, 11]

ds_ = []
ds_nl_ = []
ds_di_ = []
ds_type_ = []
iters_ = []
means_ = []
stds_ = []
times_ = []
alphas_ = []

datasets = listify(datasets)
for dsid in datasets:
    try: 
        print(f'\nProcessing {dsid} dataset...\n')
        if dsid in get_UCR_univariate_list(): scale_type=None
        else: scale_type='standardize'
        data = create_UCR_databunch(dsid, bs=2048, scale_type=scale_type, verbose=True)
        X_train = data.train_ds.x.items.astype(np.float32)
        X_valid = data.valid_ds.x.items.astype(np.float32)
        y_train = data.train_ds.y.items.astype(np.int64)
        y_valid = data.valid_ds.y.items.astype(np.int64)
    except: 
        ds_nl_.append(dsid)
        print(f'\n...{dsid} cannot be loaded\n')
        continue
    if np.isinf(X_train).sum() + np.isnan(X_train).sum() > 0: 
        ds_di_.append(dsid)
        print(f'...{dsid} contains inf or nan\n')
        continue
        
    if data.features == 1: ds_type_.append('univariate')
    else: ds_type_.append('multivariate')
    
    score_ = []
    elapsed_times_ = []
    for i in range(iterations):
        start_time = time.time()
        model = ROCKET(data.features, data.seq_len, n_kernels=n_kernels,kss=kss).to(device)
        for j, (xb,yb) in enumerate(data.train_dl.new(shuffle=False, drop_last=False)):
            with torch.no_grad(): out = model(xb)
            X_train_tfm = out if j == 0 else torch.cat((X_train_tfm, out))
        for j, (xb,yb) in enumerate(data.valid_dl.new(shuffle=False, drop_last=False)):
            with torch.no_grad(): out = model(xb)
            X_valid_tfm = out if j == 0 else torch.cat((X_valid_tfm, out))
        classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 7), normalize=True)
        classifier.fit(X_train_tfm, y_train)
        score = classifier.score(X_valid_tfm, y_valid)
        score_.append(score)
        elapsed_time = time.time() - start_time
        elapsed_times_.append(elapsed_time)
        alphas_.append(classifier.alpha_)
        print('   {:2} - score: {:.5f}  alpha: {:.2E}  time(s): {:4.0f}'.\
              format(i + 1, score, classifier.alpha_, elapsed_time))
    ds_.append(dsid)
    iters_.append(iterations)
    means_.append('{:.6f}'.format(np.mean(score_)))
    stds_.append('{:.6f}'.format(np.std(score_)))
    times_.append('{:.0f}'.format(np.mean(elapsed_times_)))
    if len(datasets) >1: clear_output()
    df = pd.DataFrame(np.stack((ds_, ds_type_, iters_,  means_, stds_, times_)).T, 
             columns=['dataset', 'type', 'iterations',  'mean_accuracy', 'std_accuracy',  'time(s)'])
    pd.set_option('display.max_columns', 999)
    pd.set_option('display.max_rows', 999)
    display(df)
    pd.set_option('display.max_columns', 20)
    pd.set_option('display.max_rows', 60)
    if ds_nl_ != []: print('\n(*) datasets not loaded      :', ds_nl_)
    if ds_di_ != []:print('(*) datasets with data issues:', ds_di_)

,dataset,type,iterations,mean_accuracy,std_accuracy,time(s)
0,ArticularyWordRecognition,multivariate,5,0.992667,0.001333,14
1,AtrialFibrillation,multivariate,5,0.306667,0.053333,13
2,BasicMotions,multivariate,5,1.000000,0.000000,13
3,Cricket,multivariate,5,1.000000,0.000000,13
4,ERing,multivariate,5,0.954074,0.002963,13
5,EigenWorms,multivariate,5,0.819847,0.013310,78
6,Epilepsy,multivariate,5,0.963768,0.000000,13
7,EthanolConcentration,multivariate,5,0.292015,0.001521,15
8,FingerMovements,multivariate,5,0.534000,0.008000,14
9,HandMovementDirection,multivariate,5,0.475676,0.026204,13



(*) datasets not loaded      : ['DuckDuckGeese', 'FaceDetection', 'InsectWingbeat']
(*) datasets with data issues: ['CharacterTrajectories', 'JapaneseVowels', 'SpokenArabicDigits']


This is the comparison with InceptionTime:

<img src="./images/ROCKET_InceptionTime_multi.png"  width="400">

## And how to use it with other classifiers? 

Once you build the 20k features per sample, you can use them to train any classifier of your choice.

If you want to use fastai, for example, with a logistic regression, you could should then build a databunch and a logistic regression model.

### Integration with fastai

In [47]:
bs = 30
scale_type='standardize'

X_train, y_train, X_valid, y_valid = get_UCR_data('OliveOil')
_, features, seq_len = X_train.shape
X_train = (X_train - X_train.mean(axis = (1, 2), keepdims = True)) / (X_train.std(axis = (1, 2), keepdims = True) + 1e-8)
X_valid = (X_valid - X_valid.mean(axis = (1, 2), keepdims = True)) / (X_valid.std(axis = (1, 2), keepdims = True) + 1e-8)
X_train_tensor = torch.tensor(X_train, device=device)
X_valid_tensor = torch.tensor(X_valid, device=device)
model = ROCKET(features, seq_len, n_kernels=10000, kss=[7, 9, 11]).to(device)
X_train_tfm = model(X_train_tensor).unsqueeze(1)
X_valid_tfm = model(X_valid_tensor).unsqueeze(1)

data = (ItemLists('.', TSList(X_train_tfm), TSList(X_valid_tfm))
        .label_from_lists(y_train, y_valid)
        .databunch(bs=min(bs, len(X_train)), val_bs=min(bs * 2, len(X_valid)))
        .scale(scale_type=scale_type))
data

TSDataBunch;

Train: LabelList (30 items)
x: TSList
TimeSeries(ch=1, seq_len=20000),TimeSeries(ch=1, seq_len=20000),TimeSeries(ch=1, seq_len=20000),TimeSeries(ch=1, seq_len=20000),TimeSeries(ch=1, seq_len=20000)
y: CategoryList
0,0,0,0,0
Path: .;

Valid: LabelList (30 items)
x: TSList
TimeSeries(ch=1, seq_len=20000),TimeSeries(ch=1, seq_len=20000),TimeSeries(ch=1, seq_len=20000),TimeSeries(ch=1, seq_len=20000),TimeSeries(ch=1, seq_len=20000)
y: CategoryList
0,0,0,0,0
Path: .;

Test: None

Let's take a look at the output of ROCKET.

In [48]:
data.show_batch()

In [50]:
model = nn.Sequential(nn.Linear(20_000, data.c))
learn = Learner(data, model, metrics=accuracy)
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,1.447277,1.838979,0.400000,00:00
1,1.636598,31.197151,0.133333,00:00
2,11.642745,31.246531,0.166667,00:00
3,16.640320,12.950564,0.166667,00:00
4,15.823500,9.460878,0.400000,00:00


### XGBoost

In [51]:
bs = 30
scale_type='standardize'

X_train, y_train, X_valid, y_valid = get_UCR_data('OliveOil')
_, features, seq_len = X_train.shape
X_train = (X_train - X_train.mean(axis = (1, 2), keepdims = True)) / (X_train.std(axis = (1, 2), keepdims = True) + 1e-8)
X_valid = (X_valid - X_valid.mean(axis = (1, 2), keepdims = True)) / (X_valid.std(axis = (1, 2), keepdims = True) + 1e-8)
X_train_tensor = torch.tensor(X_train, device=device)
X_valid_tensor = torch.tensor(X_valid, device=device)
model = ROCKET(features, seq_len, n_kernels=10000, kss=[7, 9, 11]).to(device)
X_train_tfm = model(X_train_tensor)
X_valid_tfm = model(X_valid_tensor)

In [54]:
import xgboost as xgb
classifier = xgb.XGBClassifier(max_depth=3,
                               learning_rate=0.1,
                               n_estimators=100,
                               verbosity=1,
                               objective='binary:logistic',
                               booster='gbtree',
                               tree_method='auto',
                               n_jobs=1,
                               gpu_id=0,
                               gamma=0,
                               min_child_weight=1,
                               max_delta_step=0,
                               subsample=1,
                               colsample_bytree=1,
                               colsample_bylevel=1,
                               colsample_bynode=1,
                               reg_alpha=0,
                               reg_lambda=1,
                               scale_pos_weight=1,
                               base_score=0.5,
                               random_state=0,
                               missing=None)

classifier.fit(X_train_tfm, y_train)
preds = classifier.predict(X_valid_tfm)
(preds == y_valid).mean()

0.6666666666666666

## Conclusions

ROCKET is a great method for TSC that has established a new levels of performance in terms of accuracy and time. It does it by successfully applying an approach quite different from the traditional DL approaches. The method uses 10k random kernels to generate features that are then classified by linear classifiers (although you may use a classifier of your choice).
The original method has 2 limitations (lack of multivariate and lack of GPU support) that are overcome by the Pytorch implementation shared in this notebook.